In [1]:
# https://github.com/CSBiology/deepStabP/blob/main/src/Api/requirements.txt
from fastapi import APIRouter
from pydantic import BaseModel
from transformers import  T5EncoderModel, T5Tokenizer
from tqdm.auto import *
import gc
from app.predictor import * 

In [2]:
# mirrored in dotnet Shared/DeepStabP.Types.fs
class FastaRecord(BaseModel):
    header      : str
    sequence    : str

# mirrored in dotnet Shared/DeepStabP.Types.fs
class PredictorInfo(BaseModel):
    growth_temp : int
    mt_mode     : str # "Lysate" or "Cell"
    fasta       : list[FastaRecord]

tokenizer = T5Tokenizer.from_pretrained("Rostlab/prot_t5_xl_uniref50", do_lower_case=False )
model = T5EncoderModel.from_pretrained("Rostlab/prot_t5_xl_uniref50")
gc.collect()
# https://github.com/CSBiology/deepStabP/tree/main/src/Api/trained_model/b25_sampled_10k_tuned_2_d01
prediction_net = deepSTAPpMLP.load_from_checkpoint ("trained_model/b25_sampled_10k_tuned_2_d01/checkpoints/epoch=1-step=2316.ckpt")

Some weights of the model checkpoint at Rostlab/prot_t5_xl_uniref50 were not used when initializing T5EncoderModel: ['decoder.block.12.layer.1.EncDecAttention.v.weight', 'decoder.block.9.layer.1.EncDecAttention.q.weight', 'decoder.block.4.layer.0.layer_norm.weight', 'decoder.block.8.layer.1.EncDecAttention.q.weight', 'decoder.block.11.layer.2.layer_norm.weight', 'decoder.block.21.layer.1.EncDecAttention.v.weight', 'decoder.block.15.layer.0.SelfAttention.v.weight', 'decoder.block.0.layer.0.SelfAttention.relative_attention_bias.weight', 'decoder.block.10.layer.0.SelfAttention.q.weight', 'decoder.block.22.layer.1.EncDecAttention.q.weight', 'decoder.block.19.layer.2.DenseReluDense.wi.weight', 'decoder.block.17.layer.0.SelfAttention.v.weight', 'decoder.block.9.layer.1.EncDecAttention.v.weight', 'decoder.block.5.layer.0.SelfAttention.o.weight', 'decoder.block.7.layer.0.SelfAttention.v.weight', 'decoder.block.20.layer.1.EncDecAttention.v.weight', 'decoder.block.1.layer.0.layer_norm.weight', '

In [3]:
# The processing can be found here: https://github.com/CSBiology/deepStabP/blob/main/src/Server/FastaReader.fs
# Replace 'O', 'U', 'J', 'Z', 'B' with "X" (None in this example)
SingleFastaMinimal = """>ExampleName
MAQYHQQHEMKQTMAETQYVTAPPPMGYPVMMKDSPQTVQPPHEGQSKGSGGFLRGCLAAMCCCCVLDCVF"""

fasta_record_1 = FastaRecord(header="ExampleName", sequence="MAQYHQQHEMKQTMAETQYVTAPPPMGYPVMMKDSPQTVQPPHEGQSKGSGGFLRGCLAAMCCCCVLDCVF")

predictor_info = PredictorInfo(
    growth_temp=37,
    mt_mode="Lysate",
    fasta=[fasta_record_1]
)

In [5]:
prediction = determine_tm (predictor_info.fasta, predictor_info.mt_mode, predictor_info.growth_temp, model, prediction_net, new_features, tokenizer)
prediction


c:\Users\Kevin\source\repos\deepStabP\src\Api\app\predictor.py:111: FutureWarning: The default value of regex will change from True to False in a future version.
  output_df['Protein'] = output_df['Protein'].str.replace(r"[A-Za-z][A-Za-z]\|",'')
c:\Users\Kevin\source\repos\deepStabP\src\Api\app\predictor.py:112: FutureWarning: The default value of regex will change from True to False in a future version.
  output_df['Protein'] = output_df['Protein'].str.replace(r"\|.*",'')


,Protein,Tm
0,ExampleName,61.957163


Expecting:

| index | Protein     | Melting temperature [°C] |
|-------|-------------|--------------------------|
| 0     | ExampleName | 53.441018546246454       |